# Youtube Comment Crawling

### Youtube 댓글을 크롤링하는 목적
- 예시
    - 브이로그의 댓글이 악플이 달렸는지 선플이 달렸는지 확인을 하고 싶다. -> 브이로그의 주소를 받아서 쓰면 된다
    - 축구경기를 할 때 사람들의 반을을 보고 싶다. -> 유튜브 검색창에 축구를 치고 영상에 들어가서 댓글을 크롤링하면 된다.

- 목적
    - 상황: 대형 유튜버인 내가 내 유튜브 영상에 있는 댓글들에서 악플을 찾고 싶다

- 과제 목록
    1. scroll 횟수 = 5번
    2. 댓글 수집 시작
        1. 대댓글 ❌
        2. ID / 댓글 / 좋아요 Data 크롤링
        3. Data에 video_id열 추가 (`v=video_id`)

### Youtube 크롤링을 하기 전 결정해야 할 일

- Selenium으로 만들기

### URL 파악하기
- Youtube의 도메인은 "https://www.youtube.com/"
- 내가 크롤링하고자 하는 영상의 주소는 "https://www.youtube.com/watch?v=SsJg6x-inFw"
- 키
    - `watch?v=`
        - 뒤에 영상의 고유주소

In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [34]:
# 크롬창 열기
options = Options()
driver = webdriver.Chrome(options=options)
driver.maximize_window()
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
video_id = "SsJg6x-inFw"
# 유튜브 주소 들어가기
driver.get("https://www.youtube.com/watch?v=" + video_id)



In [35]:
# 스크롤 5번하기
# 댓글창이 보이게 스크롤을 살짝 내린 뒤 스크롤 5번을 실행한다.
n_scroll = 5
ActionChains(driver).send_keys("k").perform()
last_height = driver.execute_script("return document.documentElement.scrollHeight")

driver.execute_script("window.scrollTo(0, 200);")
for i in range(5):
    
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    
    time.sleep(2)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    

# time.sleep(2)

# 페이지를 5번 스크롤
# for _ in range(n_scroll):
#     # JavaScript를 사용하여 페이지 스크롤
#     driver.execute_script("window.scrollBy(0, window.innerHeight);")
#     # 각 스크롤 후 약간의 지연을 추가하여 페이지가 로드될 시간을 줍니다.
#     time.sleep(1)
#     wait = WebDriverWait(driver, 10)
#     wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

# container=driver.find_element(By.TAG_NAME, "html")
# before_top = driver.execute_script("return arguments[0].scrollTop", container)

# for i in range(5):
#     driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", container)
#     after_top = driver.execute_script("return arguments[0].scrollTop", container)
#     time.sleep(2)
#     # if int(before_top) == int(after_top):
#     #     break
        
#     before_top = after_top 

In [5]:
# 유튜브 프리미엄 창 없애기
try:
    driver.find_element(By.CSS_SELECTOR, "#dismiss-button > a").click()
except:
    pass

In [54]:
# temp = driver.find_element(By.ID, "comments").find_element(By.ID, "contents").find_elements(By.CLASS_NAME, "style-scope.ytd-item-section-renderer")
# temp = driver.find_elements(By.ID, "context-text")
# print(temp)

# 아이디 받아오기
# y_id = temp[1].find_element(By.ID,  'author-text').text
# print(y_id)

y_id = driver.find_elements(By.CSS_SELECTOR, '#author-text')[2]
y_id.text


'@Wal_seop'

In [47]:
# # 댓글 받아오기
# y_comment = temps[i].find_element(By.ID, 'content-text').text
# print(y_comment)

y_comment = driver.find_elements(By.CSS_SELECTOR, '#content-text')
for i in range(5):
    print(y_comment[i].text)

또또또 설레발친다. 지나가던 한화팬입니다.
한화의 김경문, 롯데의 김태형.... 오랫동안 꼴찌 였던 팀들이 서서히 자리가 잡혀가고 있음. 이래서 감독이 중요한 거구나...
이거 한화 계속 본 사람들은 공감할거임.
문현빈이 롯데전 이후로 병살타로 맛이 가있었고, 하필 1루에 주자 있는데 대타 문현빈 보고 이게 뭔가 했음. 그런데 이걸 스퀴즈 번트로 역전도 해, 클러치상황만 오면 병살타를 치던 문현빈이 짜릿하게 웃고있어. 그저 대 경 문.
아니...한화가 작전야구를!?!
김경문 감독님  명장   강인권  ㅈ장  ㅋㅋㅋㅋ 땡큐 강인권  ㅋㅋㅋㅋ


In [60]:
# 좋아요 받아오기
# y_like = temp[0].find_element(By.ID, 'vote-count-middle').text
# print(y_like)

y_like = driver.find_elements(By.XPATH, '//*[@id="vote-count-middle"]')
print(len(y_like))

87


In [59]:
cnt = driver.find_elements(By.CLASS_NAME, 'style-scope ytd-comment-view-model')
print(len(cnt))

87


In [36]:
data_dict = {'ID':[],'Comments':[],'Likes':[]}
cnt = driver.find_elements(By.CLASS_NAME, 'style-scope ytd-comment-view-model')

for i in range(len(cnt)):
    y_id = driver.find_elements(By.CSS_SELECTOR, '#author-text')[i].text
    y_comment = driver.find_elements(By.CSS_SELECTOR, '#content-text')[i].text
    y_like = driver.find_elements(By.XPATH, '//*[@id="vote-count-middle"]')[i].text
    
    data_dict["ID"].append(y_id)
    data_dict["Comments"].append(y_comment)
    data_dict["Likes"].append(y_like)

y_comment_info = pd.DataFrame(data_dict)

In [37]:
y_comment_info

,ID,Comments,Likes
0,@Wal_seop,이거 한화 계속 본 사람들은 공감할거임.\n문현빈이 롯데전 이후로 병살타로 맛이 가...,598
1,@user-kb1ht2ck9t,또또또 설레발친다. 지나가던 한화팬입니다.,940
2,@friendnabe,"한화의 김경문, 롯데의 김태형.... 오랫동안 꼴찌 였던 팀들이 서서히 자리가 잡혀...",806
3,@noage622able,머 당연히 김경문 감독 몇개월 더 지켜봐야겠지만 스퀴즈 작전을 지시하고 이걸 성공시...,32
4,@Foo_Fighters_4229,아니...한화가 작전야구를!?!,237
...,...,...,...
95,@panpeter8665,"한화 선수들아,\n팬들 구단주 노감독에게 미안해서라도 죽을 힘을 다해서 뛰어라.",2
96,@CssncWidnz,근데 확실히 좀 올드한 감독들이 스몰야구 모티브였던 세대라 작전이 재밋음,1
97,@user-bv6zf9ih5r,와… 5승 2패 1무 한화 팬이지만 ㅈㄴ 감동스럽다,1
98,@user-qf7gl9br4s,한화는 우승하면 모든 타팬들도\n축하해줄듯,1


In [38]:
y_comment_info["Video_id"] = video_id

y_comment_info.set_index("Video_id")

,ID,Comments,Likes
Video_id,,,
SsJg6x-inFw,@Wal_seop,이거 한화 계속 본 사람들은 공감할거임.\n문현빈이 롯데전 이후로 병살타로 맛이 가...,598
SsJg6x-inFw,@user-kb1ht2ck9t,또또또 설레발친다. 지나가던 한화팬입니다.,940
SsJg6x-inFw,@friendnabe,"한화의 김경문, 롯데의 김태형.... 오랫동안 꼴찌 였던 팀들이 서서히 자리가 잡혀...",806
SsJg6x-inFw,@noage622able,머 당연히 김경문 감독 몇개월 더 지켜봐야겠지만 스퀴즈 작전을 지시하고 이걸 성공시...,32
SsJg6x-inFw,@Foo_Fighters_4229,아니...한화가 작전야구를!?!,237
...,...,...,...
SsJg6x-inFw,@panpeter8665,"한화 선수들아,\n팬들 구단주 노감독에게 미안해서라도 죽을 힘을 다해서 뛰어라.",2
SsJg6x-inFw,@CssncWidnz,근데 확실히 좀 올드한 감독들이 스몰야구 모티브였던 세대라 작전이 재밋음,1
SsJg6x-inFw,@user-bv6zf9ih5r,와… 5승 2패 1무 한화 팬이지만 ㅈㄴ 감동스럽다,1
